In [1]:
import glob
import re
import pandas as pd
from IPython.core.display import display

In [2]:
coref_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Coref_Clusters'])
print(coref_df)

Empty DataFrame
Columns: [Doc_ID, Text_ID, Text, Coref_Clusters]
Index: []


In [7]:
def get_coref_clusters(path, document_id):
    temp_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Coref_Clusters'])

    with open(path, 'r') as file:
        lines = file.readlines()

        text_id = 0
        text = []
        coref_clusters = {}
        temp_clusters = {}

        for line in lines:
            if line == '\n':
                temp_df = temp_df.append({'Doc_ID': document_id,
                                          'Text_ID': document_id + '-' + text_id,
                                          'Text': text,
                                          'Coref_Clusters': list(coref_clusters.values())},
                                         ignore_index=True)
                text = []
                coref_clusters = {}
                temp_clusters = {}
            elif not line.startswith('#'):
                columns = line.split()
                text_id = columns[1]
                text.append(columns[3])
                coref_column = columns[12]
                index = int(columns[2])
                if not coref_column == '-':
                    corefs = coref_column.split('|')
                    for coref in corefs:
                        key = coref.strip('()')
                        if re.match(r'\([0-9]+\)', coref):
                            coref_clusters.setdefault(key, []).append([index])
                        if re.match(r'^\([0-9]+$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                        if re.match(r'^[0-9]+\)$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                            cluster_span = temp_clusters.pop(key)
                            # borde ändra till [temp_clusters.pop(key), temp_clusters.pop(key)]
                            coref_clusters.setdefault(key, []).append(cluster_span)
    return temp_df

In [8]:
for path in glob.iglob('data/datasets/suc-core-mod/' + 'aa*.conll'):
    filename = path.split('/')[-1]
    document_id = filename.split('-')[0]
    df = get_coref_clusters(path, document_id)
    coref_df = coref_df.append(df)

In [12]:
display(coref_df)
coref_df.to_pickle('data/coref_small.pkl')

,Doc_ID,Text_ID,Text,Coref_Clusters
0,aa09,aa09-001,"[Alla, vet, ju, ändå, hur, det, går]",[]
1,aa09,aa09-002,"[Svagt, intresse, för, det, tjeckoslovakiska, ...","[[[4, 6]]]"
2,aa09,aa09-003,"[PRAG, (, TT:s, utsända, )]",[[[1]]]
3,aa09,aa09-004,"[Fem, dagar, före, de, första, fria, valen, i,...","[[[9]], [[4, 12]], [[17]]]"
4,aa09,aa09-005,"[Husväggarna, runt, Vaclav-platsen, är, visser...","[[[3]], [[11, 15]]]"
...,...,...,...,...
132,aa05,aa05-039,"[-, Det, kan, bana, vägen, för, en, helt, ny, ...",[]
133,aa05,aa05-040,"[Alla, är, dock, inte, lika, optimistiska, .]",[]
134,aa05,aa05-041,"[-, Vad, skall, de, betala, med, ?]",[[[4]]]
135,aa05,aa05-042,"[De, har, ju, ingen, hårdvaluta, ,, säger, tex...","[[[1]], [[9, 10], [12, 17]]]"
